In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import casadi as ca

---
# Part 1: Explicit ODE Model

The explicit API is best when you have state-space form:
$$\dot{x} = f(x, u, p, t)$$

## Simple Pendulum Equations

$$\dot{\theta} = \omega$$
$$\dot{\omega} = -\frac{g}{l} \sin(\theta)$$

In [ ]:
from cyecca.dynamics.explicit import Model, explicit, state, param, output_var, input_var

@explicit
class SimplePendulum:
    """Simple pendulum with no damping."""
    # States: x(t) - differentiated variables
    theta: float = state(desc="angle from vertical [rad]")
    omega: float = state(desc="angular velocity [rad/s]")
    
    # Parameters: p - constants
    l: float = param(default=1.0, desc="pendulum length [m]")
    g: float = param(default=9.81, desc="gravity [m/s^2]")
    
    # Outputs: y(t) - algebraic (computed from states)
    x_pos: float = output_var(desc="horizontal position [m]")
    y_pos: float = output_var(desc="vertical position [m]")
    energy: float = output_var(desc="total mechanical energy [J]")


def create_simple_pendulum():
    """Create and build the simple pendulum model."""
    model = Model(SimplePendulum)
    v = model.v  # Symbolic namespace
    
    # Define dynamics: ẋ = f_x(x, u, p, t)
    model.ode(v.theta, v.omega)                           # dθ/dt = ω
    model.ode(v.omega, -v.g / v.l * ca.sin(v.theta))      # dω/dt = -g/l * sin(θ)
    
    # Define outputs: y = f_y(x, u, p, t)
    model.output(v.x_pos, v.l * ca.sin(v.theta))          # x = l*sin(θ)
    model.output(v.y_pos, -v.l * ca.cos(v.theta))         # y = -l*cos(θ)
    
    # Energy: E = 1/2 * m * l^2 * ω^2 + m * g * l * (1 - cos(θ))  [assuming m=1]
    kinetic = 0.5 * v.l**2 * v.omega**2
    potential = v.g * v.l * (1 - ca.cos(v.theta))
    model.output(v.energy, kinetic + potential)
    
    model.build()
    return model


# Create model
pendulum = create_simple_pendulum()
print(f"Model: {pendulum.name}")
print(f"States: {pendulum.state_names}")
print(f"Outputs: {pendulum.output_names}")
print(f"\nDynamics function f_x: {pendulum.f_x}")
print(f"Output function f_y: {pendulum.f_y}")

In [ ]:
# Set initial conditions using typed views (autocomplete works!)
pendulum.v0.theta = 0.5  # Initial angle: ~29 degrees
pendulum.v0.omega = 0.0  # Start from rest

# Or use the filtered view:
pendulum.x0.theta = 0.5  # Same as above, but x0 only shows states

print("Initial state (v0):")
print(f"  theta = {pendulum.v0.theta:.3f} rad ({np.degrees(pendulum.v0.theta):.1f}°)")
print(f"  omega = {pendulum.v0.omega:.3f} rad/s")

print("\nParameters (p0):")
print(f"  l = {pendulum.p0.l:.2f} m")
print(f"  g = {pendulum.p0.g:.2f} m/s²")

In [ ]:
# Simulate
t, data = pendulum.simulate(t0=0.0, tf=10.0, dt=0.01)

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Angle
axes[0, 0].plot(t, np.degrees(data.theta), 'b-', linewidth=1.5)
axes[0, 0].set_xlabel('Time [s]')
axes[0, 0].set_ylabel('θ [degrees]')
axes[0, 0].set_title('Pendulum Angle')
axes[0, 0].grid(True, alpha=0.3)

# Angular velocity
axes[0, 1].plot(t, data.omega, 'r-', linewidth=1.5)
axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('ω [rad/s]')
axes[0, 1].set_title('Angular Velocity')
axes[0, 1].grid(True, alpha=0.3)

# Phase portrait
axes[1, 0].plot(np.degrees(data.theta), data.omega, 'g-', linewidth=1.5)
axes[1, 0].set_xlabel('θ [degrees]')
axes[1, 0].set_ylabel('ω [rad/s]')
axes[1, 0].set_title('Phase Portrait')
axes[1, 0].grid(True, alpha=0.3)

# Energy conservation
axes[1, 1].plot(t, data.energy, 'm-', linewidth=1.5)
axes[1, 1].set_xlabel('Time [s]')
axes[1, 1].set_ylabel('Energy [J]')
axes[1, 1].set_title('Total Energy (should be conserved)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Energy variation: {data.energy.max() - data.energy.min():.6f} J (should be ~0)")

In [ ]:
# Animate the pendulum motion
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 0.5)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_title('Pendulum Trajectory')

# Plot trajectory
ax.plot(data.x_pos, data.y_pos, 'b-', alpha=0.3, linewidth=1)

# Plot a few snapshots
n_snapshots = 10
indices = np.linspace(0, len(t)-1, n_snapshots, dtype=int)
colors = plt.cm.viridis(np.linspace(0, 1, n_snapshots))

for i, idx in enumerate(indices):
    x, y = data.x_pos[idx], data.y_pos[idx]
    ax.plot([0, x], [0, y], 'o-', color=colors[i], markersize=8, linewidth=2)
    
ax.plot(0, 0, 'ko', markersize=10)  # Pivot point
plt.show()

---
## Linearization

Linearize around the equilibrium point (θ=0, ω=0):

$$\dot{x} \approx A \cdot \delta x + B \cdot \delta u$$

For small angles: $\ddot{\theta} \approx -\frac{g}{l} \theta$

In [ ]:
# Linearize at equilibrium
pendulum.v0.theta = 0.0
pendulum.v0.omega = 0.0

A, B, C, D = pendulum.linearize()

print("State matrix A:")
print(A)
print(f"\nExpected: [[0, 1], [-g/l, 0]] = [[0, 1], [{-pendulum.p0.g/pendulum.p0.l:.2f}, 0]]")

# Eigenvalue analysis
eigenvalues = np.linalg.eigvals(A)
print(f"\nEigenvalues: {eigenvalues}")

# Natural frequency
omega_n = np.sqrt(pendulum.p0.g / pendulum.p0.l)
print(f"\nNatural frequency: ω_n = √(g/l) = {omega_n:.3f} rad/s")
print(f"Period: T = 2π/ω_n = {2*np.pi/omega_n:.3f} s")

---
# Part 2: Damped Pendulum with Input

Add damping and external torque input:

$$\dot{\theta} = \omega$$
$$\dot{\omega} = -\frac{g}{l} \sin(\theta) - c \omega + \frac{\tau}{ml^2}$$

In [ ]:
@explicit
class DampedPendulum:
    """Damped pendulum with external torque input."""
    # States
    theta: float = state(desc="angle [rad]")
    omega: float = state(desc="angular velocity [rad/s]")
    
    # Inputs
    tau: float = input_var(default=0.0, desc="external torque [N·m]")
    
    # Parameters  
    l: float = param(default=1.0, desc="length [m]")
    m: float = param(default=1.0, desc="mass [kg]")
    g: float = param(default=9.81, desc="gravity [m/s²]")
    c: float = param(default=0.5, desc="damping coefficient [1/s]")


def create_damped_pendulum():
    model = Model(DampedPendulum)
    v = model.v
    
    # Dynamics with damping and input torque
    model.ode(v.theta, v.omega)
    model.ode(v.omega, -v.g/v.l * ca.sin(v.theta) - v.c * v.omega + v.tau / (v.m * v.l**2))
    
    model.build()
    return model


damped = create_damped_pendulum()
damped.v0.theta = 1.0  # Start at ~57 degrees
damped.v0.omega = 0.0

# Simulate free response (no input)
t, data = damped.simulate(0.0, 10.0, 0.01)

plt.figure(figsize=(10, 4))
plt.plot(t, np.degrees(data.theta), 'b-', linewidth=1.5)
plt.xlabel('Time [s]')
plt.ylabel('θ [degrees]')
plt.title('Damped Pendulum - Free Response')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Simulate with time-varying input (sinusoidal forcing)
damped.v0.theta = 0.0
damped.v0.omega = 0.0

def forcing_torque(t, model):
    """Sinusoidal forcing at natural frequency."""
    omega_n = np.sqrt(model.p0.g / model.p0.l)
    return {'tau': 0.5 * np.sin(omega_n * t)}

t, data = damped.simulate(0.0, 20.0, 0.01, u_func=forcing_torque)

fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

axes[0].plot(t, np.degrees(data.theta), 'b-', linewidth=1.5)
axes[0].set_ylabel('θ [degrees]')
axes[0].set_title('Forced Damped Pendulum (resonance)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(t, data.tau, 'r-', linewidth=1.5)
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('τ [N·m]')
axes[1].set_title('Applied Torque')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# Part 3: Implicit DAE Model

The implicit API is best when:
- Variable types are inferred from `.dot()` usage
- You want a more Modelica-like equation syntax

Variables with `.dot()` called become **states**, others become **algebraic**.

In [ ]:
from cyecca.dynamics.implicit import Model as ImplicitModel, implicit, var, param as iparam

@implicit
class ImplicitPendulum:
    """Pendulum using implicit DAE formulation.
    
    Variables become states automatically when .dot() is called.
    """
    # Variables - type inferred from .dot() usage
    theta: float = var()  # Will become STATE (has .dot() in equations)
    omega: float = var()  # Will become STATE (has .dot() in equations)
    
    # Parameters
    l: float = iparam(default=1.0)
    g: float = iparam(default=9.81)


def create_implicit_pendulum():
    model = ImplicitModel(ImplicitPendulum)
    v = model.v
    
    # Define equations in residual form: 0 = f(...)
    # Using .dot() automatically marks variables as states!
    model.eq(v.theta.dot() - v.omega)                      # 0 = dθ/dt - ω
    model.eq(v.omega.dot() + v.g/v.l * ca.sin(v.theta))    # 0 = dω/dt + g/l·sin(θ)
    
    model.build()
    return model


implicit_pendulum = create_implicit_pendulum()
print(f"Model: {implicit_pendulum.name}")
print(f"States (inferred from .dot()): {implicit_pendulum.state_names}")
print(f"Algebraic variables: {implicit_pendulum.algebraic_names}")

In [ ]:
# Simulate the implicit model
implicit_pendulum.v0.theta = 0.5
implicit_pendulum.v0.omega = 0.0

t, data = implicit_pendulum.simulate(0.0, 10.0, 0.01)

plt.figure(figsize=(10, 4))
plt.plot(t, np.degrees(data.theta), 'b-', linewidth=1.5, label='Implicit Model')
plt.xlabel('Time [s]')
plt.ylabel('θ [degrees]')
plt.title('Implicit DAE Pendulum')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---
# Part 4: Comparison - Explicit vs Implicit

Both formulations should give identical results.

In [ ]:
# Create both models with same parameters
explicit_model = create_simple_pendulum()
implicit_model = create_implicit_pendulum()

# Same initial conditions
theta0 = 0.8
explicit_model.v0.theta = theta0
explicit_model.v0.omega = 0.0

implicit_model.v0.theta = theta0
implicit_model.v0.omega = 0.0

# Simulate both
t_exp, data_exp = explicit_model.simulate(0.0, 10.0, 0.01)
t_imp, data_imp = implicit_model.simulate(0.0, 10.0, 0.01)

# Compare
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

axes[0].plot(t_exp, np.degrees(data_exp.theta), 'b-', linewidth=2, label='Explicit')
axes[0].plot(t_imp, np.degrees(data_imp.theta), 'r--', linewidth=2, label='Implicit')
axes[0].set_ylabel('θ [degrees]')
axes[0].set_title('Comparison: Explicit vs Implicit Model')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Error
error = np.degrees(data_exp.theta - data_imp.theta)
axes[1].plot(t_exp, error, 'g-', linewidth=1.5)
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('Difference [degrees]')
axes[1].set_title('Numerical Difference')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Max difference: {np.max(np.abs(error)):.6f} degrees")

---
# Summary

## When to use Explicit ODE (`cyecca.dynamics.explicit`)

- Standard state-space form: $\dot{x} = f(x, u, p, t)$
- Clear separation of states, inputs, parameters, outputs
- Control system design, linearization
- Typed field descriptors: `state()`, `input_var()`, `param()`, `output_var()`

## When to use Implicit DAE (`cyecca.dynamics.implicit`)

- Natural equation form (like Modelica)
- Variable types inferred from `.dot()` usage
- Mixed differential-algebraic systems
- Field descriptors: `var()`, `param()`

## Key Features

| Feature | Explicit | Implicit |
|---------|----------|----------|
| Variable types | Declared explicitly | Inferred from `.dot()` |
| Equation form | `model.ode(var, expr)` | `model.eq(residual)` |
| State access | `model.x0.name` | `model.v0.name` |
| Dynamics function | `model.f_x` | `model.f_x` |
| Output function | `model.f_y` | N/A |